In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     |████████████████████████████████| 61kB 1.7MB/s 
Requirement already up-to-date: google-auth-httplib2 in /usr/local/lib/python3.6/dist-packages (0.0.4)
Requirement already up-to-date: google-auth-oauthlib in /usr/local/lib/python3.6/dist-packages (0.4.1)
     |████████████████████████████████| 92kB 4.5MB/s 
ERROR: google-api-core 1.22.4 has requirement google-auth<2.0dev,>=1.21.1, but you'll have google-auth 1.17.2 which is incompatible.
  Found existing installation: google-api-core 1.16.0
    Uninstalling google-api-core-1.16.0:
      Successfully uninstalled google-api-core-1.16.0
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


# ***Sending Email and Generating Folder***

In [1]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# ***Helper Functions***

In [2]:
def callback(request_id, response, exception):
    if exception:
        # Handle error
        print(exception)
    else:
        print("Permission Id: %s" % response.get('id'))


# ***Sending Email***

In [3]:
Id = '1620652042'
email = 'Abiduzzaman.khan@northsouth.edu'
parent = '1N9pulzZSKSgT8SCK_FA2x0jKH_KWmcZ5'
page_token = None
folderToCreate = ['picture','video']
while True:
    #getting the list of all the folders
    response = drive_service.files().list(q = "'1N9pulzZSKSgT8SCK_FA2x0jKH_KWmcZ5' in parents and mimeType='application/vnd.google-apps.folder' and trashed = false",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
    for file in response.get('files', []):
        if Id == file.get('name').split('_')[0]:
          print("ID already present")
          break
    else:
        # creating student folder
        file_metadata = {
            'name': Id,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents' : [parent]
        }
        folder = drive_service.files().create(body=file_metadata,
                                            fields='id').execute()
        # creating folders inside student folder      
        for folderName in folderToCreate:
                  file_metadata = {
                        'name': folderName,
                        'mimeType': 'application/vnd.google-apps.folder',
                        'parents' : [folder.get('id')]
                    }

                  drive_service.files().create(body=file_metadata,
                                            fields='id').execute()

        batch = drive_service.new_batch_http_request(callback=callback)
        user_permission = {
            'type': 'user',
            'role': 'writer',
            'emailAddress': email
            
        }
        batch.add(drive_service.permissions().create(
                fileId=folder.get('id'),
                body=user_permission,
                fields='id',

        ))
        batch.execute()
        
    page_token = response.get('nextPageToken', None)
    if page_token is None:
        break

ID already present
